### 0-1 Knapsack Problem

In this prolem, we are given a set of items, each with a weight and a value, and we need to determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.

The items are indivisible; we can either take an item or not (0-1 property) For example:
```
Input:
value = [20, 5, 10, 40, 15, 25]
weight = [1, 2, 3, 8, 7, 4]
int W = 10

Output: Knapsack value is 60

value = 20 + 40
weight = 1 + 8 = 9 < W
```

We could just use recursion and try out all the possibilities. If we have the room, try adding it or not adding it, always keeping track of what our maximum value is.

In [1]:
def knapsackRec(values, weights, max_weight, curr_val=0, max_val=0, item=0):
    if item >= len(values):
        return max_val
    added = None
    if max_weight - weights[item] >= 0:
        added_val = curr_val + values[item]
        added_max_val = added_val if added_val > max_val else max_val
        added = knapsackRec(values, weights, max_weight-weights[item], added_val, added_max_val, item+1)
    skipped = knapsackRec(values, weights, max_weight, curr_val, max_val, item+1)
    return skipped if added == None else max(added, skipped)

In [2]:
# a little bit cleaner, fewer parameters
def knapsackRec2(values, weights, max_weight, item=0):
    if item >= len(values):
        return 0
    added = None
    if max_weight - weights[item] >= 0:
        added = values[item] + knapsackRec2(values, weights, max_weight-weights[item], item+1)
    skipped = knapsackRec2(values, weights, max_weight, item+1)
    return skipped if added == None else max(added, skipped)

In [3]:
values = [20, 5, 10, 40, 15, 25]
weights = [1, 2, 3, 8, 7, 4]
W = 10
values2 = [20, 5, 10, 40, 15, 25, 30, 50, 22, 8, 17, 3, 42, 10, 15, 32]
weights2 = [1, 2, 3, 8, 7, 4, 23, 18, 9, 10, 5, 2, 8, 11, 25, 30]
W2 = 55

In [4]:
%%time
knapsackRec(values, weights, W)

CPU times: user 61 µs, sys: 1 µs, total: 62 µs
Wall time: 67.2 µs


60

In [5]:
%%time
knapsackRec(values2, weights2, W2)

CPU times: user 27.1 ms, sys: 2.42 ms, total: 29.6 ms
Wall time: 131 ms


221

In [6]:
%%time
knapsackRec2(values, weights, W)

CPU times: user 58 µs, sys: 1 µs, total: 59 µs
Wall time: 62.9 µs


60

In [7]:
%%time
knapsackRec2(values2, weights2, W2)

CPU times: user 26.3 ms, sys: 2.21 ms, total: 28.5 ms
Wall time: 50.4 ms


221

### Memoized version

Keep a dictionary that tracks max values for each item we're looking at a given weight.

In [19]:
def knapsackMemo(values, weights, max_weight, item=0, lookup=None):
    if item >= len(values):
        return 0
    if lookup == None:
        lookup = {}
    key = f'{item}-{max_weight}'
    if lookup.get(key):
        return lookup[key]
    added = None
    if max_weight - weights[item] >= 0:
        added = values[item] + knapsackMemo(values, weights, max_weight-weights[item], item+1, lookup)
    skipped = knapsackMemo(values, weights, max_weight, item+1, lookup)
    if added == None:
        lookup[key] = skipped
    else:
        lookup[key] = max(added, skipped)
    return lookup[key]

In [20]:
%%time
knapsackMemo(values2, weights2, W2)

CPU times: user 1.99 ms, sys: 67 µs, total: 2.06 ms
Wall time: 2.21 ms


221

### Lookup table

To solve this problem in a bottom-up manner, we solve smaller subproblems first, then solve larger subproblems from there. We solve the maximum value that can be attained with all weights possible and with first i items, using the values of those already computed.

Example:

Values = [20, 5, 10, 40, 15, 25]
Weights = [1, 2, 3, 8, 7, 4]
Max weight = 10

We set up 0s for if the value had 0 value and if the allowed weight was 0.

From there we fill in the matrix. At (1, 1) we'll check the previous weight possible (1 - weight of the item) with the previous item, so we can see what our previous value is if we're including the item. Also check the current weight with the previous item, so we can check what our value is if we're excluding the item. Add the value of the item to our first value we looked up and compare to the second value and accept whatever is higher.
```
For (1, 1): weights[1-1] = 1, 1-1 = 0, (0, 0) = 0, values[0] = 20, 0 + 20 = 20. (1, 0) = 0. 20 > 0

For (3, 2) : weights[2-1] = 2, 3-2 = 1, (1, 2) = 20, values[1] = 5, 20 + 5 = 25. (3, 1) = 20. 25 > 20
 

                      0       20       5       10       40       15       25     
             0        0        0       0        0        0        0        0
             1        0       20      20       20       20       20       20
             2        0       20      20       20       20       20       20
             3        0       20      25       25       25       25       25
             4        0       20      25       30       30       30       30
             5        0       20      25       30       30       30       45
             6        0       20      25       35       35       35       45
             7        0       20      25       35       35       35       45
             8        0       20      25       35       40       40       60
             9        0       20      25       35       60       60       60
             10       0       20      25       35       60       60       60

```

In [35]:
def knapsackMatrix(values, weights, max_weight):
    lookup = [[0] * (len(values)+1) for _ in range(max_weight + 1)]
    for v in range(1, len(values) + 1):
        for w in range(1, max_weight + 1):
            value = values[v-1]
            weight = weights[v-1]
            prev_weight = w - weight
            prev_weight_value = None
            if prev_weight >= 0:
                prev_weight_value = lookup[prev_weight][v-1]
            prev_curr_weight = lookup[w][v-1]
            lookup[w][v] = prev_curr_weight if prev_weight_value == None else max(prev_curr_weight, prev_weight_value + value)
    return lookup[-1][-1]

In [37]:
%%time
knapsackMatrix(values, weights, W)

CPU times: user 98 µs, sys: 1 µs, total: 99 µs
Wall time: 103 µs


60

In [38]:
%%time
knapsackMatrix(values2, weights2, W2)

CPU times: user 1.14 ms, sys: 19 µs, total: 1.16 ms
Wall time: 1.2 ms


221